#### Save monthly AWRA PET data

In [4]:
%who

agcd_info	 alldata_dict	 awra_info	 chirps_info	 get_da	 gleam_info	 glob	 mswep_info	 np	 
out_dir	 save_monthly_data	 sys	 xr	 


In [1]:
%run -i drought_2017to2020/alldata_info.py

out_dir = '/g/data/w97/ad9701/drought_2017to2020/PminusPET/'

In [2]:
alldata_dict['awra']

{'full_path': '/g/data/fj8/BoM/AWRA/DATA/SCHEDULED-V6/',
 'file_name': 'e0_*.nc',
 'varname': 'e0',
 'lat_slice': slice(-10, -44, None),
 'lon_slice': slice(112, 154, None),
 'units': 'mm month-1',
 'rename_latlon': True}

In [3]:
save_monthly_data(alldata_dict = alldata_dict, data_name = 'awra', out_dir = out_dir, calc_from_daily = True)

#### Calculating P - PET

In [3]:
%who

ds_P	 main_dir	 np	 xr	 


In [2]:
import xarray as xr
import numpy as np

main_dir = '/g/data/w97/ad9701/drought_2017to2020/'
ds_P = xr.open_dataset(main_dir + 'precip_data/agcd_monthly_1900_2020.nc')

In [4]:
ds_PET = xr.open_dataset(main_dir + 'PminusPET/e0_awra_monthly_1911_2022.nc')
lat_new = np.float32(ds_PET['lat'])
lon_new = np.float32(ds_PET['lon'])
ds_PET = ds_PET.assign_coords({'lat': lat_new, 'lon': lon_new})
ds_PET = ds_PET.reindex(lat=ds_PET.lat[::-1])

In [5]:
import pandas as pd
time_new_pet = pd.to_datetime(ds_PET['time']).to_period('M').to_timestamp()
ds_PET = ds_PET.assign_coords({'time': time_new_pet})

time_new_p = pd.to_datetime(ds_P['time']).to_period('M').to_timestamp()
ds_P = ds_P.assign_coords({'time': time_new_p})

In [8]:
# ds_P['precip'].load()
# ds_PET['e0'].load()
da_PminusPET = ds_P['precip'] - ds_PET['e0']
da_PminusPET.rename('PminusPET').to_netcdf(main_dir + 'PminusPET/PminusPET_awra_monthly_1911_2020.nc')

### Scratch Space

In [17]:
ds_PET

<xarray.Dataset>
Dimensions:  (time: 1335, lat: 681, lon: 841)
Coordinates:
  * time     (time) datetime64[ns] 1911-01-31 1911-02-28 ... 2022-03-31
  * lat      (lat) float32 -10.0 -10.05 -10.1 -10.15 ... -43.9 -43.95 -44.0
  * lon      (lon) float32 112.0 112.1 112.1 112.2 ... 153.9 153.9 153.9 154.0
Data variables:
    e0       (time, lat, lon) float64 ...

In [18]:
lat1 = ds_P.lat.values
lat2 = np.flip(ds_PET.lat.values)

In [19]:
np.max(lat1 - lat2)

0.0

### Scratch Space

In [5]:
out_dir = '/g/data/w97/ad9701/drought_2017to2020/'
save_monthly_data(alldata_dict = alldata_dict, data_name = 'gleam', out_dir = out_dir)

In [6]:
DA = get_da(alldata_dict, 'awra')

In [9]:
test = DA.resample(time="M").sum()

In [15]:
test

<xarray.DataArray 'e0' (time: 1335, lat: 681, lon: 841)>
dask.array<stack, shape=(1335, 681, 841), dtype=float64, chunksize=(1, 681, 841), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) datetime64[ns] 1911-01-31 1911-02-28 ... 2022-03-31
  * lat      (lat) float64 -10.0 -10.05 -10.1 -10.15 ... -43.9 -43.95 -44.0
  * lon      (lon) float64 112.0 112.0 112.1 112.2 ... 153.8 153.9 153.9 154.0

In [11]:
test.time.values

array(['1911-01-31T00:00:00.000000000', '1911-02-28T00:00:00.000000000',
       '1911-03-31T00:00:00.000000000', ...,
       '2022-01-31T00:00:00.000000000', '2022-02-28T00:00:00.000000000',
       '2022-03-31T00:00:00.000000000'], dtype='datetime64[ns]')

In [12]:
ds_P = xr.open_dataset('/g/data/w97/ad9701/drought_2017to2020/precip_data/agcd_monthly_1900_2020.nc')
ds_P

<xarray.Dataset>
Dimensions:  (time: 1445, lat: 681, lon: 841)
Coordinates:
  * time     (time) datetime64[ns] 1900-01-16 1900-02-14T12:00:00 ... 2020-05-16
  * lat      (lat) float32 -44.0 -43.95 -43.9 -43.85 ... -10.1 -10.05 -10.0
  * lon      (lon) float32 112.0 112.1 112.1 112.2 ... 153.9 153.9 153.9 154.0
Data variables:
    precip   (time, lat, lon) float32 ...

In [14]:
ds_P['precip'].time

<xarray.DataArray 'time' (time: 1445)>
array(['1900-01-16T00:00:00.000000000', '1900-02-14T12:00:00.000000000',
       '1900-03-16T00:00:00.000000000', ..., '2020-03-16T00:00:00.000000000',
       '2020-04-15T12:00:00.000000000', '2020-05-16T00:00:00.000000000'],
      dtype='datetime64[ns]')
Coordinates:
  * time     (time) datetime64[ns] 1900-01-16 1900-02-14T12:00:00 ... 2020-05-16
Attributes:
    axis:           T
    standard_name:  time
    time_zone:      local time
    bounds:         time_bnds
    long_name:      time